In [17]:
import csv
import pandas as pd
import numpy as np
import math 
import sklearn
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split
# !pip3 install tensorflow
import tensorflow as tf
from tensorflow import feature_column
from tensorflow.keras import layers

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LogisticRegression
from verstack.stratified_continuous_split import scsplit # pip install verstack
from sklearn.model_selection import train_test_split

import sys
import pickle
from pickle import dump
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from collections import Counter
import seaborn as sns
import joblib
from sklearn.utils import shuffle
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import xgboost
from sklearn.ensemble import VotingRegressor


import time

dataframe = pd.read_csv('data/train.csv')
dataframe['user_verified'] = dataframe['user_verified'].astype(int)
dataframe.head()

dataframe = dataframe.drop(columns=['id', 'user_mentions', 'urls', 'hashtags', 'text'])

train, val = train_test_split(dataframe, test_size=0.25)

# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
        dataframe = dataframe.copy()
        labels = dataframe.pop('retweet_count')
        ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
        if shuffle:
                ds = ds.shuffle(buffer_size=len(dataframe))
        ds = ds.batch(batch_size)
        return ds

x_train = df_to_dataset(dataframe = train)
x_eval = df_to_dataset(dataframe = val)

feature_columns = []

# numeric cols
for header in ['timestamp', 'user_verified', 'user_statuses_count', 'user_followers_count', 'user_friends_count']:
        feature_columns.append(feature_column.numeric_column(header))


feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

model = tf.keras.Sequential([feature_layer, layers.Dense(64, activation='relu'), layers.Dense(128, activation='relu'),layers.Dense(64, activation='relu'),layers.Dense(32, activation='relu'),layers.Dense(16, activation='relu'),  layers.Dropout(.1), layers.Dense(1)])

model.compile(optimizer='adam', loss=tf.keras.losses.MeanAbsoluteError(), metrics=['mean_absolute_error'])

model.fit(x_train, validation_data=x_eval, epochs=3, use_multiprocessing=True)

print(model.evaluate(x_train))
loss, accuracy = model.evaluate(x_eval)
print("Loss, Accuracy", loss, accuracy)


Epoch 1/3
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
15558/15605 [============================>.] - ETA: 0s - loss: 38809136.0000 - mean_absolute_error: 38809136.0000WARNING:tensorflow:Layers in a Sequential model should only have a single input tensor, but we receive a <class 'dict'> input: {'timestamp': <tf.Tensor 'ExpandDims:0' shape=(None, 1) dtype=int64>, 'user_verified': <tf.Tensor 'ExpandDims_4:0' shape=(None, 1) dtype=int64>, 'user_statuses_count': <tf.Tensor 'ExpandDims_3:0' shape=(None, 1) dtype=int64>, 'user_followers_count': <tf.Tensor 'ExpandDims_1:0' shape=(None, 1) dtype=int64>, 'user_friends_count': <tf.Tensor 'ExpandDims_2:0' shape=(None, 1) dtype=int64>}
Consider rewriting this model with the Functional API.
15605/15605 [==============================] - 13s 846us/step - loss: 38694420.0000 - mean_absolute_error: 38694420.0000 - val_loss: 146.3066 - val_mean_absolute_error: 146.3066
Epoch 2/3
15605/156